In [ ]:
import numpy as np
import torch
import pandas as pd
from datasets import ClassLabel, Dataset
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader

num_classes = 2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_version = 'xlm-roberta-large'

data = pd.read_csv('../new_data.csv')
test = Dataset.from_pandas(pd.DataFrame({'text': data["text"], 'label': data["label"]}))

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_version)
model = AutoModelForSequenceClassification.from_pretrained(model_version, num_labels=num_classes).to(device)

In [ ]:
def tokenize_batch(batch):
    tokens = tokenizer(batch['text'], padding="max_length", truncation=True, max_length=512)
   # tokens['label'] = labels.str2int(batch['label'])
    return tokens

test_tokenized = test.map(tokenize_batch, batched=True, batch_size=len(test))
test_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

eval_dataloader = DataLoader(test_tokenized, batch_size=8)

In [ ]:
models_to_load = ["roberta_large_v1_0779.pth"]

for i, model_name in enumerate(models_to_load):
    model.load_state_dict(torch.load(f"models/{model_name}"))

    pred_probas = np.empty((0, 2))

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        y_pred_proba = torch.softmax(logits, axis=1).cpu().numpy()
        pred_probas = np.vstack((pred_probas, y_pred_proba))

    if i == 0:
        all_preds = y_pred_proba
    else:
        all_preds += y_pred_proba